In [ ]:
!pip install tigramite==5.2.5.8 numba==0.58.1 numpy==1.26.4 llvmlite==0.41.1

In [ ]:
import os
import numpy as np
import pickle
from safetensors.torch import load_file

import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.lpcmci import LPCMCI
from tigramite.independence_tests.parcorr import ParCorr
import matplotlib.pyplot as plt

In [ ]:
dist_type = "gaussian"
window_sizes_to_run = [450, 300, 150]
seeds_to_run = [42, 123, 2024]
causal_algorithms = {'PCMCI': PCMCI, 'LPCMCI': LPCMCI}
all_causal_results_parcorr = {}

output_results_dir = f"../result/causal_results_{dist_type}_parcorr"
output_images_dir = f"../result/causal_graphs_{dist_type}_parcorr"
os.makedirs(output_results_dir, exist_ok=True)
os.makedirs(output_images_dir, exist_ok=True)

In [ ]:
activities = ['Walk', 'Run', 'Jump', 'Sitting', 'Empty Room']

def load_latent_data(latent_space_dir, window_size, seed, dist_type, activity_name):
    all_features = []
    num_antennas = 4
    for antenna_idx in range(num_antennas):
        dist_tag = dist_type[:4]
        filename = f"{dist_tag}_win{window_size}_ant{antenna_idx}_seed{seed}.safetensors"
        path = os.path.join(latent_space_dir, filename)
        if not os.path.exists(path):
            raise FileNotFoundError(f"File not found: {path}")
        
        loaded_tensors = load_file(path)
        
        activity_index = activities.index(activity_name)
        labels_np = loaded_tensors["labels"].numpy()
        indices = np.where(labels_np == activity_index)[0]
        
        features = loaded_tensors["latent_space"].numpy()[indices]
        all_features.append(features)
        
    fused_features = np.concatenate(all_features, axis=1)
    return fused_features

In [ ]:

for window_size in window_sizes_to_run:
    all_causal_results_parcorr[window_size] = {}
    for seed in seeds_to_run:
        all_causal_results_parcorr[window_size][seed] = {}
        for activity in activities:
            print(f"--- Processing: Gaussian (ParCorr), Win={window_size}, Seed={seed}, Activity={activity} ---")
            try:
                activity_graph_dir = os.path.join(output_images_dir, f"window_{window_size}", activity.replace(' ', '_'))
                os.makedirs(activity_graph_dir, exist_ok=True)

                latent_space_dir = f"../dataset/latent_space_dataset_{dist_type}"
                latent_vectors = load_latent_data(latent_space_dir, window_size, seed, dist_type, activity)
            
                    
                num_features = latent_vectors.shape[1]
                proposition_names = [f'LF_{i}' for i in range(num_features)]
                
                dataframe = pp.DataFrame(latent_vectors, var_names=proposition_names)
                cond_ind_test = ParCorr() 
                
                results_per_algo = {}
                for algo_name, algo_class in causal_algorithms.items():
                    print(f"  > Running {algo_name}...")
                    causal_discovery = algo_class(dataframe=dataframe, cond_ind_test=cond_ind_test, verbosity=0)
                    
                    if algo_name == 'PCMCI':
                        results = causal_discovery.run_pcmci(tau_max=3, pc_alpha=0.01)
                    else:
                        results = causal_discovery.run_lpcmci(tau_min=1, tau_max=3, pc_alpha=0.01)
                    
                    results_per_algo[algo_name] = results
                    
                    fig, ax = tp.plot_time_series_graph(
                        figsize=(10, 8), val_matrix=results['val_matrix'],
                        graph=results['graph'], var_names=proposition_names,
                        link_colorbar_label='ParCorr Coeff.', cmap_edges="viridis"
                    )
                    fig.suptitle(f'Causal Graph ({algo_name}) for {activity}\nGaussian, win={window_size}, seed={seed}', y=1.02)
                    
                    image_filename = f"gaus_seed{seed}_{algo_name}.png"
                    full_save_path = os.path.join(activity_graph_dir, image_filename)
                    fig.savefig(full_save_path, bbox_inches='tight')
                    plt.close(fig)
                    print(f"  > Causal graph for {algo_name} saved.")
                
                all_causal_results_parcorr[window_size][seed][activity] = results_per_algo

            except Exception as e:
                print(f"  > An error occurred: {e}\n")


results_filepath = os.path.join(output_results_dir, 'all_causal_results_parcorr.pkl')
with open(results_filepath, 'wb') as f:
    pickle.dump(all_causal_results_parcorr, f)
print(f"\n--- Gaussian (ParCorr) experiments processed! Results saved to: {results_filepath} ---")